# Websocket 

## Praktischer Teil - Chatbot

**1.Auftrag**

Das Konzept der Chat-Bot Applikation soll im Kontext des erarbeiteten Themas implementiert werden. Die Chat Applikation soll zeigen, wie eine einfache Implementierung dieses Themenbereichs aussehen kann und welche Funktionalitäten, aber auch welche Probleme und Schwierigkeiten damit verbunden sind. Das Konzept der Applikation kann auf die Eigenschaften der Technologie/Protokoll/Patterns angepasst werden. Es soll auch Wert auf die Integration des kvanC Jupyter Systems gelegt werden. Das heisst, dass die IP Adressen der Studierenden (oder die Usernamen via /etc/hosts) wenn angebracht oder nötig verwendet werden. Ebenso soll die Chat Applikation (evtl auch nur der Client) in das Jupyter Notebook miteinbezogen werden und so (nach einer Verteilung der Notebooks) von allen Studierenden ausprobiert werden können. Dies kann auch durch die Nutzung des Terminals erreicht werden, falls andere Programmiersprachen verwendet werden.

### Steps to do
Write a coroutine that handles a single connection. It receives a WebSocket protocol instance and the URI path in argument.
- Call recv() and send() to receive and send messages at any time.
- When recv() or send() raises ConnectionClosed, clean up and exit. If you started other asyncio.Task, terminate them before exiting.
- If you aren’t awaiting recv(), consider awaiting wait_closed() to detect quickly when the connection is closed.
- You may ping() or pong() if you wish but it isn’t needed in general.

Create a server with serve() which is similar to asyncio’s create_server(). You can also use it as an asynchronous context manager.
- The server takes care of establishing connections, then lets the handler execute the application logic, and finally closes the connection after the handler exits normally or with an exception.
- For advanced customization, you may subclass WebSocketServerProtocol and pass either this subclass or a factory function as the create_protocol argument.

## Server

### WebSocket Example Server

In [1]:
# imports
import websockets
import asyncio

import nest_asyncio
nest_asyncio.apply()

In [2]:
# Init connection and create new websocket
host = 'julia.mueller'
port = 2000

In [3]:
# send message and receive it
async def receiveMessage(message):
    print(f"10. < {message}")

async def sendMessage():
    print("sendMessage called.")
    message = f"Hello Client, your message is received!!"
    await asyncio.sleep(2) # sleep for 2 seconds before returning message
    print("returning message and exiting sendMessage\n")
    return message
    

In [4]:
# handler for receiving and sending messages
async def receiveMessage_handler(websocket):
    print('6. entering receiveMessage_handler')
    async for msg in websocket:
        print('9. calling for receiveMessage')
        await receiveMessage(msg)
        print('11. Message received')
        print('12. exiting receiveMessage_handler \n')

        
async def sendMessage_handler(websocket):
    print("7. entering sendMessage_handler:\n")
    while True:
        print('8. waiting for message')
        message = await sendMessage()
        print('13. Message before sending')
        await websocket.send(message)
        print('14. message sent.')
        
# sending/receiving combined in one handler
async def message_handler(websocket):
    print('3. calling receive_task')
    receive_task = asyncio.ensure_future(
        receiveMessage_handler(websocket))
    print('4. calling send_task')
    send_task = asyncio.ensure_future(
        sendMessage_handler(websocket))
    print('5. wait for done and pending \n')
    done, pending = await asyncio.wait(
        [receive_task, send_task],
        return_when=asyncio.FIRST_COMPLETED,
    )
    for task in pending:
        task.cancel()

In [5]:
# Create broadcast-communication
clientsConnected = set()

async def broadcast():
    while True:
        await asyncio.gather(
            *[websocket.send("Hello, this is a broadcast communication. Welcome!") for websocket in clientsConnected],
            return_exceptions=False
        )
        await asyncio.sleep(3)
        
asyncio.create_task(broadcast())
        
async def broadcast_handler(websocket):
    print(websocket)
    clientsConnected.add(websocket)
    try:
        async for msg in websocket:
            #websocket.send()
            pass
    except websockets.ConnectionClosedError:
        pass
    finally:
        clientsConnected.remove(websocket)

In [6]:
# example from https://websockets.readthedocs.io/en/stable/intro.html
async def receiver(websocket, path):
    print(f"2. message_handler is called \n")
    await message_handler(websocket)
    #await broadcast_handler(websocket)

start_server = websockets.serve(receiver, host, port)
print(f"1. Websocket-Server {start_server} started.")

loop = asyncio.get_event_loop()
loop.run_until_complete(start_server)

1. Websocket-Server <websockets.server.Serve object at 0x7fa88c617d00> started.


2. message_handler is called 

2. message_handler is called 

